In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from PIL import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

DATA_DIR = '/kaggle/input/plant-pathology-2020-fgvc7'
SEED = 1279

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Initial Data Analysis

In [ ]:
!pip install ImageHash

In [ ]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
train_df.rename(columns={'multiple_diseases': 'multi'}, inplace=True)
to_drop = ['Train_379', 'Train_782', 'Train_1661'] # dropping repeated/mislabeled samples
train_df = train_df[~train_df.image_id.isin(to_drop)]
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
print(train_df.head())
print(test_df.head())
print(type(train_df.iloc[0]))
for iterable in train_df.iloc[0].index:
    print(iterable)

In [ ]:
# image display stuff

def disp(img_row):
    img = Image.open(f"{DATA_DIR}/images/{img_row['image_id']}.jpg")
    plt.figure()
    plt.axis('off')
    plt.title(', '.join([f'{lbl}={img_row[lbl]}' for lbl in img_row.index]))
    plt.imshow(img)

In [ ]:
rusty_leaves = train_df[train_df['rust'] > 0]
idxs = np.random.choice(len(rusty_leaves), size=10)
for idx in idxs:
    disp(rusty_leaves.iloc[idx])

In [ ]:
scabby = train_df[train_df['scab'] > 0]
idxs = np.random.choice(len(scabby), size=10)
for idx in idxs:
    disp(scabby.iloc[idx])

In [ ]:
helth = train_df[train_df['healthy'] > 0]
idxs = np.random.choice(len(helth), size=10)
for idx in idxs:
    disp(helth.iloc[idx])

In [ ]:
idxs = np.random.choice(len(test_df), size=10)
for i in idxs:
    disp(test_df.iloc[i])

In [ ]:
# looking for similar images (https://github.com/JohannesBuchner/imagehash/blob/master/find_similar_images.py)
import imagehash
def find_similar_images(userpaths, hashfunc=imagehash.phash):
    def is_image(filename):
        f = filename.lower()
        return f.endswith(".png") or f.endswith(".jpg") or \
            f.endswith(".jpeg") or f.endswith(".bmp") or \
            f.endswith(".gif") or '.jpg' in f or  f.endswith(".svg")
    
    image_filenames = []
    for userpath in userpaths:
        image_filenames += [os.path.join(userpath, path) for path in os.listdir(userpath) if is_image(path)]
    images = {}
    problematic = []
    print("filenames obtained")
    for img in tqdm(sorted(image_filenames)):
        try:
            hash = hashfunc(Image.open(img))
        except Exception as e:
            print('Problem:', e, 'with', img)
            continue
        if hash in images:
            print(img, '  already exists as', ' '.join(images[hash]))
            problematic += [img, *images[hash]]
#             if 'dupPictures' in img:
#                 print('rm -v', img)
        images[hash] = images.get(hash, []) + [img]
    return problematic
    
problematic = find_similar_images([f'{DATA_DIR}/images/'])

In [ ]:
# print(problematic)
pairs = []
for k in range(0,len(problematic),2):
    pairs.append( (problematic[k].rsplit('/', 1)[-1], problematic[k+1].rsplit('/', 1)[-1]) )
print(pairs[-3:])

In [ ]:
idxs = [379, 1173, 782, 592, 815, 1661]
for i in idxs:
    disp(train_df.iloc[i])
    
# seems like we should drop Train_379, Train_782, and Train_1661

In [ ]:
for col in ['healthy', 'multi', 'rust', 'scab']:
    print(f"{col}, {sum(train_df[col])}")
    
# data distribution: fairly even except for less multi

## Model Training
Note: This is designed to be runnable even if the data analysis part is not run. As such, there's some code repeat.

In [2]:
from torch.utils.data.dataset import Dataset, Subset
from torch.utils.data import DataLoader
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from sklearn.metrics import roc_auc_score

### Utility Functions

In [3]:
def get_train_data():
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df.rename(columns={'multiple_diseases': 'multi'}, inplace=True)
    to_drop = ['Train_379', 'Train_782', 'Train_1661'] # dropping repeated/mislabeled samples
    train_df = train_df[~train_df.image_id.isin(to_drop)]
    files = [f"{DATA_DIR}/images/{fname}.jpg" for fname in train_df['image_id']]
    labels = train_df.iloc[:, 1:].to_numpy().argmax(axis=1).astype(int)
    return files, labels

In [4]:
def get_test_data():
    test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
    files = [f"{DATA_DIR}/images/{fname}.jpg" for fname in test_df['image_id']]
    return files, None

In [5]:
def get_transforms(train=True):
    transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((224, 224)), # change this to some variable later!
            torchvision.transforms.ToTensor()
    ])
    return transforms

In [6]:
def compute_acc(outputs, labels):
    return np.mean(outputs==labels)

def compute_mean_auc(outputs, labels):
    return roc_auc_score(labels, outputs, average='macro', multi_class='ovr')

### Dataset and Dataloader setup

In [7]:
class AppleDataset(Dataset):
    def __init__(self, filepaths, transforms, labels=None):
        self.transforms = transforms
        self.filepaths = filepaths
        self.labels = labels
        
    def __len__(self):
        return len(self.filepaths)
    
    def __getitem__(self, i):
        img = Image.open(self.filepaths[i]).convert('RGB')
        img = self.transforms(img)
        lbl = self.labels[i] if self.labels is not None else None
        return (img, lbl)

In [8]:
# Lightning folds !
class AppleKFoldDataModule(pl.LightningDataModule):
    def __init__(self, k_folds=5):
        self.k_folds = k_folds
        super().__init__()
    
    def setup(self, stage=None):
        transforms = get_transforms()
        train_files, train_labels = get_train_data()
        test_files = get_test_data()
        self.train_dset = AppleDataset(train_files, transforms, labels=train_labels)
        self.test_dset = AppleDataset(test_files, transforms)
    
    def setup_folds(self):
        rng = np.random.default_rng(seed=SEED)
        shuffled_idx = rng.permutation(len(self.train_dset))
        self.splits = np.array_split(shuffled_idx, self.k_folds)
        
    def setup_fold_index(self, i):
        train_idx = np.concatenate(self.splits[:i] + self.splits[i+1:])
        val_idx = self.splits[i]
        self.train_fold = Subset(self.train_dset, train_idx)
        self.val_fold = Subset(self.train_dset, val_idx)
    
    def train_dataloader(self, bsz=32):
        return DataLoader(self.train_fold, batch_size=bsz, shuffle=True) # change batch size for variable
    
    def val_dataloader(self, bsz=32):
        return DataLoader(self.val_fold, batch_size=bsz, shuffle=False)
        
    def test_dataloader(self, bsz=32):
        return DataLoader(self.test_dset, batch_size=bsz, shuffle=False)

### Model(s) Setup

In [9]:
def initialize_base_model(model_name, pretrain=True):
    if model_name == 'resnet-50':
        model = torchvision.models.resnet50(pretrained=pretrain)
        inp_size = 224
        out_size = 1000
        
    return model, inp_size, out_size

In [10]:
class GenericAppleArch(nn.Module):
    def __init__(self, base_model_name='resnet-50', out_classes=4):
        super(GenericAppleArch, self).__init__()
        self.ft_extractor, self.inp_size, out_size = initialize_base_model(base_model_name, pretrain=True)
        self.clfr = nn.Sequential(nn.Linear(out_size, out_classes), nn.Softmax(dim=1))
        
    def forward(self, x):
        x = self.ft_extractor(x)
        out = self.clfr(x)
        return out
    

class AppleModel(pl.LightningModule): #Lightning-fied
    def __init__(self, base_model_name='resnet-50', out_classes=4, lr=1e-5):
        super().__init__()
        self.model = GenericAppleArch(base_model_name=base_model_name, out_classes=out_classes)
        self.lr = lr
        self.save_hyperparameters()
        
    def on_train_epoch_start(self):
        self.train_outputs = []
        self.train_ys = []
    
    def on_validation_epoch_start(self):
        self.val_outputs = []
        self.val_ys = []
        
    def training_step(self, batch, batch_idx):
        x,y = batch
        out = self.model(x)
        loss = F.nll_loss(out, y)
        self.train_outputs += list(out.detach().cpu().numpy())
        self.train_ys += list(y.detach().cpu())
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        out = self.model(x)
        self.val_outputs += list(out.detach().cpu().numpy())
        self.val_ys += list(y.detach().cpu().numpy())
        loss = F.nll_loss(out, y)
        self.log('val_loss', loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        return optimizer
    
    def on_train_epoch_end(self):
        all_outputs = np.array(self.train_outputs)
        all_preds = np.argmax(all_outputs, axis=1)
        all_ys = np.array(self.train_ys)
        epoch_acc = compute_acc(all_preds, all_ys)
        mean_auc = compute_mean_auc(all_outputs, all_ys)
        self.log_dict({'train_acc': epoch_acc, 'train_mean_auc': mean_auc})
        print(f"Train acc={epoch_acc}")
        print(f"Train mean auc={mean_auc}")
    
    def on_validation_epoch_end(self):
        all_outputs = np.array(self.val_outputs)
        all_preds = np.argmax(all_outputs, axis=1)
        all_ys = np.array(self.val_ys)
        epoch_acc = compute_acc(all_preds, all_ys)
        mean_auc = compute_mean_auc(all_outputs, all_ys)
        self.log_dict({'val_acc': epoch_acc, 'val_mean_auc': mean_auc})
        print(f"Val acc={epoch_acc}")
        print(f"Val mean auc={mean_auc}")

### Training Loop

In [11]:
from copy import deepcopy
from datetime import datetime
from pytz import timezone
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [12]:
## TODO: WANDB
# https://wandb.ai/manan-goel/MNIST/reports/How-to-Integrate-PyTorch-Lightning-with-Weights-Biases--VmlldzoxNjg1ODQ1
import wandb
from pytorch_lightning.loggers import WandbLogger
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
pl.seed_everything(SEED)
exp_name = "baseline"
try:
    wandb.finish()
except:
    pass
k_models = []
config = {
    'trainer_kwargs': 
        {
            'max_epochs': 30,
            'accelerator': "gpu" if torch.cuda.is_available() else "cpu",
            'default_root_dir': "/kaggle/working/",
            'auto_lr_find': False,
            'log_every_n_steps': 10,
            'callbacks': [EarlyStopping(monitor="val_loss", mode="min")]
        },
    'model_kwargs':
        {
            'base_model_name': 'resnet-50', 
            'out_classes': 4,
            'lr': 1e-5
        }
}
dm = AppleKFoldDataModule(k_folds=5)
dm.prepare_data()
dm.setup()
dm.setup_folds()
model = AppleModel()
base_state_dict = model.state_dict()
for k in range(dm.k_folds):
    config['trainer_kwargs']['logger'] = WandbLogger(project="apples", name=f"{exp_name}_fold{k+1}of{dm.k_folds}")
    trainer = pl.Trainer(**config['trainer_kwargs'])
    dm.setup_fold_index(k)
    model.load_state_dict(base_state_dict)
    trainer.fit(model, datamodule=dm)
    time = datetime.now(timezone('US/Pacific')).strftime("%m_%d-h%H_m%M")
    save_filepath = f"{exp_name}-fold{k+1}of{dm.k_folds}-{time}"
    trainer.save_checkpoint(f"{save_filepath}.ckpt")
    wandb.finish()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

wandb: Currently logged in as: cpolzak. Use `wandb login --relogin` to force relogin


Sanity Checking: 0it [00:00, ?it/s]

Val acc=0.0625
Val mean auc=0.7151078453636417


Training: 0it [00:00, ?it/s]

In [20]:
wandb.finish()

Error in callback <function _WandbInit._resume_backend at 0x7fd0995867a0> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7fd099586830> (for post_run_cell):


Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 152, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 138, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface_shared.py", line 405, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface_shared.py", line 226, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/opt/cond

Exception: The wandb backend process has shutdown

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 170, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 127, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface_shared.py", line 395, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface_shared.py", line 226, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/opt/conda/lib/python3.7/site